In [2]:
from keras import backend as K
from keras.models import load_model
from keras.optimizers import Adam
from scipy.misc import imread
import numpy as np
from matplotlib import pyplot as plt

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization
from data_generator.object_detection_2d_data_generator import DataGenerator
from eval_utils.average_precision_evaluator import Evaluator

%matplotlib inline

C:\Users\c1yeung\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\c1yeung\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.CSRRD7HKRKC3T3YXA7VY7TAZGLSWDKW6.gfortran-win_amd64.dll
C:\Users\c1yeung\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
  stacklevel=1)
Using TensorFlow backend.


In [3]:
# Set a few configuration parameters.
img_height = 300
img_width = 300
n_classes = 20
model_mode = 'inference'

In [4]:
# # 1: Build the Keras model

K.clear_session() # Clear previous models from memory.

model = ssd_300(image_size=(img_height, img_width, 3),
                n_classes=n_classes,
                mode=model_mode,
                l2_regularization=0.0005,
                scales=[0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05], # The scales for MS COCO [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05]
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
                two_boxes_for_ar1=True,
                steps=[8, 16, 32, 64, 100, 300],
                offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                clip_boxes=False,
                variances=[0.1, 0.1, 0.2, 0.2],
                normalize_coords=True,
                subtract_mean=[123, 117, 104],
                swap_channels=[2, 1, 0],
                confidence_thresh=0.01,
                iou_threshold=0.45,
                top_k=200,
                nms_max_output_size=400)

# 2: Load the trained weights into the model.

weights_path = 'VGG_coco_SSD_300x300_iter_400000_subsampled_20_classes.h5'
model.load_weights(weights_path, by_name=True)

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [14]:
dataset = DataGenerator()

# TODO: Set the paths to the dataset here.
Pascal_VOC_dataset_images_dir = './VOC2007/VOC2007/JPEGImages/'
Pascal_VOC_dataset_annotations_dir = './VOC2007/VOC2007/Annotations/'
Pascal_VOC_dataset_image_set_filename = './VOC2007/VOC2007/ImageSets/Main/test.txt'

# The XML parser needs to now what object class names to look for and in which order to map them to integers.
classes = ['background','person','bird','cat','cow','dog','horse','sheep','aeroplane','bicycle',
          'boat','bus','car','motorbike','train','bottle','chair','diningtable','pottedplant','sofa','tvmonitor']

dataset.parse_xml(images_dirs=[Pascal_VOC_dataset_images_dir],
                  image_set_filenames=[Pascal_VOC_dataset_image_set_filename],
                  annotations_dirs=[Pascal_VOC_dataset_annotations_dir],
                  classes=classes,
                  include_classes='all',
                  exclude_truncated=False,
                  exclude_difficult=False,
                  ret=False)



Processing image set 'test.txt':   0%|                                                         | 0/619 [00:00<?, ?it/s]

Processing image set 'test.txt':   5%|██▍                                            | 32/619 [00:00<00:01, 316.84it/s]

Processing image set 'test.txt':  11%|█████                                          | 66/619 [00:00<00:01, 320.71it/s]

Processing image set 'test.txt':  15%|███████                                        | 93/619 [00:00<00:01, 301.57it/s]

Processing image set 'test.txt':  20%|█████████▏                                    | 123/619 [00:00<00:01, 299.30it/s]

Processing image set 'test.txt':  26%|███████████▋                                  | 158/619 [00:00<00:01, 312.06it/s]

Processing image set 'test.txt':  31%|██████████████                                | 190/619 [00:00<00:01, 312.56it/s]

Processing image set 'test.txt':  36%|████████████████▍                             | 221/619 [00:00<00:01, 308.99it/s]

Processing image set 'test.txt

In [15]:
evaluator = Evaluator(model=model,
                      n_classes=n_classes,
                      data_generator=dataset,
                      model_mode=model_mode)

results = evaluator(img_height=img_height,
                    img_width=img_width,
                    batch_size=32,
                    data_generator_mode='resize',
                    round_confidences=False,
                    matching_iou_threshold=0.5,
                    border_pixels='include',
                    sorting_algorithm='quicksort',
                    average_precision_mode='sample',
                    num_recall_points=11,
                    ignore_neutral_boxes=True,
                    return_precisions=True,
                    return_recalls=True,
                    return_average_precisions=True,
                    verbose=True)

mean_average_precision, average_precisions, precisions, recalls = results

Number of images in the evaluation dataset: 619



  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

Producing predictions batch-wise:   0%|                                                         | 0/20 [00:00<?, ?it/s]

Producing predictions batch-wise:   5%|██▍                                              | 1/20 [00:28<09:08, 28.84s/it]

Producing predictions batch-wise:  10%|████▉                                            | 2/20 [00:57<08:38, 28.78s/it]

Producing predictions batch-wise:  15%|███████▎                                         | 3/20 [01:22<07:51, 27.72s/it]

Producing predictions batch-wise:  20%|█████████▊                                       | 4/20 [01:48<07:13, 27.07s/it]

Producing predictions batch-wise:  25%|████████████▎                                    | 5/20 [02:12<06:35, 26.35s/it]

Producing predictions batch-wise:  30%|██████████████▋                                  | 6/20 [02:39<

  0%|                                                                                          | 0/721 [00:00<?, ?it/s]

Matching predictions to ground truth, class 20/20.:   0%|                                      | 0/721 [00:00<?, ?it/s]

Matching predictions to ground truth, class 20/20.: 100%|█████████████████████████| 721/721 [00:00<00:00, 18025.66it/s]Computing precisions and recalls, class 1/20
Computing precisions and recalls, class 2/20
Computing precisions and recalls, class 3/20
Computing precisions and recalls, class 4/20
Computing precisions and recalls, class 5/20
Computing precisions and recalls, class 6/20
Computing precisions and recalls, class 7/20
Computing precisions and recalls, class 8/20
Computing precisions and recalls, class 9/20
Computing precisions and recalls, class 10/20
Computing precisions and recalls, class 11/20
Computing precisions and recalls, class 12/20
Computing precisions and recalls, class 13/20
Computing precisions and recalls, class 14/20
Comp

In [16]:
for i in range(1, len(average_precisions)):
    print("{:<14}{:<6}{}".format(classes[i], 'AP', round(average_precisions[i], 3)))
print()
print("{:<14}{:<6}{}".format('','mAP', round(mean_average_precision, 3)))

person        AP    0.669
bird          AP    0.698
cat           AP    0.896
cow           AP    0.686
dog           AP    0.847
horse         AP    0.77
sheep         AP    0.701
aeroplane     AP    0.806
bicycle       AP    0.803
boat          AP    0.657
bus           AP    0.75
car           AP    0.725
motorbike     AP    0.658
train         AP    0.936
bottle        AP    0.43
chair         AP    0.536
diningtable   AP    0.856
pottedplant   AP    0.454
sofa          AP    0.784
tvmonitor     AP    0.685

              mAP   0.717
